In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

Google Colab Integration

In [ ]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Global Configuration

In [ ]:
DATABASE_FILE_PATH = '/content/drive/MyDrive/crypto_data.db'
COINS_IN_PORTFOLIO = ['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'ADAUSDT', 'XRPUSDT', 'DOGEUSDT']
MARKET_BENCHMARK = 'BTCUSDT'
BENCHMARK_PORTFOLIO_RULE = 'risk_level'

In [ ]:
RISK_THRESHOLDS = {
    'Volatility': {'limit': 5.0, 'operator': '<='},
    'Sharpe Ratio': {'limit': 1.0, 'operator': '>='},
    'Max Drawdown': {'limit': -20.0, 'operator': '>='},
    'Sortino Ratio': {'limit': 1.0, 'operator': '>='},
    'Beta': {'limit': 1.2, 'operator': '<='},
    'Max Asset Weight': {'limit': 0.40, 'operator': '<='}
}

Email Alert Configuration

In [ ]:
SENDER_EMAIL = 'gowthamslm2004@gmail.com'
RECEIVER_EMAIL = 'gowthamsrinivasan2004@gmail.com'
try:
    SENDER_PASSWORD = userdata.get('GMAIL_APP_PASSWORD')
except userdata.SecretNotFoundError:
    print("WARNING: Secret 'GMAIL_APP_PASSWORD' not found. Email alerts will fail.")
    SENDER_PASSWORD = None

ANALYSIS & RISK CHECKING ENGINE

In [ ]:
def assign_weights(investment_goal, assets):
    weights = {}
    if investment_goal == 'risk_level':
        capped_weight = 0.40
        weights['BTCUSDT'] = capped_weight
        excess_weight = 0.50 - capped_weight
        original_other_weights = {'ETHUSDT': 0.25, 'SOLUSDT': 0.10, 'ADAUSDT': 0.05, 'XRPUSDT': 0.05, 'DOGEUSDT': 0.05}
        total_other_weight = sum(original_other_weights.values())
        for asset, weight in original_other_weights.items():
            proportion = weight / total_other_weight
            weights[asset] = weight + (excess_weight * proportion)
    else:
        return assign_weights('risk_level', assets)

    total_weight = sum(weights.values())
    if not np.isclose(total_weight, 1.0):
        weights = {asset: w / total_weight for asset, w in weights.items()}
    return weights

In [ ]:
def calculate_volatility(daily_returns):
  return daily_returns.std()

In [ ]:
def calculate_sharpe_ratio(daily_returns):
    if daily_returns.std() == 0: return 0.0
    return (daily_returns.mean() / daily_returns.std()) * np.sqrt(365)

In [ ]:
def calculate_max_drawdown(prices):
    cumulative_returns = (1 + prices / 100).cumprod()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    return drawdown.min() * 100

In [ ]:
def calculate_sortino_ratio(daily_returns):
    negative_returns = daily_returns[daily_returns < 0]
    downside_deviation = negative_returns.std()
    if np.isnan(downside_deviation) or downside_deviation == 0: return np.inf
    return (daily_returns.mean() / downside_deviation) * np.sqrt(365)

In [ ]:
def calculate_beta(portfolio_returns, market_returns):
    covariance = np.cov(portfolio_returns, market_returns)[0, 1]
    market_variance = np.var(market_returns)
    if market_variance == 0: return 0.0
    return covariance / market_variance

In [ ]:
def check_max_asset_weight(weights):
  return max(weights.values())

In [ ]:
def run_risk_checker():
    print("--- RUNNING ANALYSIS AND RISK CHECKS ---")
    conn = sqlite3.connect(DATABASE_FILE_PATH)
    cur = conn.cursor()

    cur.execute("CREATE TABLE IF NOT EXISTS risk_assessment (id INTEGER PRIMARY KEY, portfolio_name TEXT, rule_name TEXT, calculated_value REAL, rule_check TEXT, status TEXT, timestamp TEXT)")

    try:
        df = pd.read_sql_query("SELECT Date, Symbol, Close FROM crypto_prices ORDER BY Date ASC", conn, parse_dates=['Date'])
    except pd.io.sql.DatabaseError as e:
        print(f"FATAL ERROR: Could not read 'crypto_prices' table. Reason: {e}")
        conn.close()
        return []
    conn.close()

    prices_df = df.pivot(index='Date', columns='Symbol', values='Close')
    returns_df = prices_df.pct_change() * 100

    portfolio_name = BENCHMARK_PORTFOLIO_RULE
    print(f"Analyzing benchmark portfolio: '{portfolio_name}' with 40% weight constraint...")
    weights = assign_weights(portfolio_name, COINS_IN_PORTFOLIO)

    portfolio_returns = (returns_df[list(weights.keys())] * pd.Series(weights)).sum(axis=1).dropna()
    market_returns = returns_df[MARKET_BENCHMARK].dropna()
    common_index = portfolio_returns.index.intersection(market_returns.index)

    metrics = {
        'Volatility': calculate_volatility(portfolio_returns),
        'Sharpe Ratio': calculate_sharpe_ratio(portfolio_returns / 100),
        'Max Drawdown': calculate_max_drawdown(portfolio_returns),
        'Sortino Ratio': calculate_sortino_ratio(portfolio_returns / 100),
        'Beta': calculate_beta(portfolio_returns[common_index], market_returns[common_index]),
        'Max Asset Weight': check_max_asset_weight(weights)
    }

    results = []
    failed_rules = []
    for rule_name, value in metrics.items():
        threshold = RISK_THRESHOLDS[rule_name]['limit']
        operator = RISK_THRESHOLDS[rule_name]['operator']
        passed = eval(f"{value} {operator} {threshold}")
        status = "PASS" if passed else "FAIL"
        result_row = {'portfolio_name': portfolio_name, 'rule_name': rule_name, 'calculated_value': round(value, 4), 'rule_check': f"{operator} {threshold}", 'status': status, 'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        results.append(result_row)
        if not passed: failed_rules.append(result_row)

    print("\n--- Risk Assessment Summary ---")
    for res in results:
        print(f"Portfolio: {res['portfolio_name']}, Rule: {res['rule_name']:<18}, Status: {res['status']:<4}, Value: {res['calculated_value']:<8} (Rule: {res['rule_check']})")

    return results, failed_rules

In [ ]:
def store_risk_results_in_db(results):
    if not results:
        print("No results to store in the database.")
        return

    print("\n--- Storing risk check results in the database... ---")
    conn = sqlite3.connect(DATABASE_FILE_PATH)
    cur = conn.cursor()

    # Use the name of the portfolio from the first result
    name_to_clear = results[0]['portfolio_name']
    cur.execute("DELETE FROM risk_assessment WHERE portfolio_name = ?", (name_to_clear,))

    data_to_insert = [tuple(res.values()) for res in results]
    cur.executemany("INSERT INTO risk_assessment (portfolio_name, rule_name, calculated_value, rule_check, status, timestamp) VALUES (?, ?, ?, ?, ?, ?)", data_to_insert)

    conn.commit()
    conn.close()
    print("Successfully stored risk assessment results.")

In [ ]:
def send_email_alert(failed_rules):
    """Formats and sends an email alert with the details of failed rules."""
    if SENDER_PASSWORD is None:
        print("\nCannot send email alert: Password not configured in Colab Secrets.")
        return
    print("\nA risk rule has failed. Preparing to send email alert...")
    subject = "ALERT: Portfolio Risk Rule Violation Detected"
    body = "The following risk rules have failed for the monitored portfolio:\n\n"
    for failure in failed_rules:
        body += (f"  - Portfolio: {failure['portfolio_name']}\n"
                 f"    Rule: {failure['rule_name']}\n"
                 f"    Condition: {failure['rule_check']}\n"
                 f"    Actual Value: {failure['calculated_value']}\n\n")
    msg = MIMEMultipart()
    msg['From'], msg['To'], msg['Subject'] = SENDER_EMAIL, RECEIVER_EMAIL, subject
    msg.attach(MIMEText(body, 'plain'))
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.send_message(msg)
        print("Email alert sent successfully.")
    except Exception as e:
        print(f"  > ERROR: Failed to send email. Reason: {e}")

MAIN EXECUTION BLOCK


In [ ]:
if __name__ == "__main__":

    all_results, failed_rules_list = run_risk_checker()

    store_risk_results_in_db(all_results)

    if failed_rules_list:
        send_email_alert(failed_rules_list)
    else:
        print("\nAll risk rules passed. No alert necessary.")

    print("\n--- RISK CHECKER COMPLETE ---")

--- RUNNING ANALYSIS AND RISK CHECKS ---
Analyzing benchmark portfolio: 'risk_level' with 40% weight constraint...

--- Risk Assessment Summary ---
Portfolio: risk_level, Rule: Volatility        , Status: PASS, Value: 3.2857   (Rule: <= 5.0)
Portfolio: risk_level, Rule: Sharpe Ratio      , Status: FAIL, Value: 0.7105   (Rule: >= 1.0)
Portfolio: risk_level, Rule: Max Drawdown      , Status: FAIL, Value: -79.4214 (Rule: >= -20.0)
Portfolio: risk_level, Rule: Sortino Ratio     , Status: FAIL, Value: 0.9983   (Rule: >= 1.0)
Portfolio: risk_level, Rule: Beta              , Status: PASS, Value: 1.089    (Rule: <= 1.2)
Portfolio: risk_level, Rule: Max Asset Weight  , Status: PASS, Value: 0.4      (Rule: <= 0.4)

--- Storing risk check results in the database... ---
Successfully stored risk assessment results.

A risk rule has failed. Preparing to send email alert...
Email alert sent successfully.

--- RISK CHECKER COMPLETE ---
